In [1]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.memory.chat_memory import ChatMessageHistory
from langchain.memory.chat_message_histories import RedisChatMessageHistory
from langchain import OpenAI, LLMChain
from langchain.chat_models import ChatOpenAI

import configparser, os
config = configparser.ConfigParser()
config.read('./keys.ini')
openai_api_key = config['OPENAI']['OPENAI_API_KEY']

from agent_tools import tools
from agents import *

In [2]:
message_history = RedisChatMessageHistory(url='redis://localhost:6379/0', ttl=600, session_id='buffer')
memory = ConversationBufferMemory(memory_key="chat_history", chat_memory=message_history)

prefix = """You are PRTS / 普瑞赛斯, an AI assistant to answer questions about the Arknigths story by querying vector database. 
"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "chat_history", "agent_scratchpad"]
)

llm_chain = LLMChain(llm=ChatOpenAI(temperature=0.4,model_name="gpt-3.5-turbo-16k"), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)

In [3]:
agent_chain("output Near Light")



> Entering new AgentExecutor chain...
Thought: I should output the content of the "Near Light" chapter from the Arknights story.
Action: output chapter content to html
Action Input: "Near Light"
Observation: success: write query to output.html
Thought:I have successfully outputted the content of the "Near Light" chapter to an HTML file.
Final Answer: The content of the "Near Light" chapter has been outputted to an HTML file.

> Finished chain.


{'input': 'output Near Light',
 'chat_history': '',
 'output': 'The content of the "Near Light" chapter has been outputted to an HTML file.'}

In [3]:
agent_chain("find all stories about Twin Empresses?")



> Entering new AgentExecutor chain...
Thought: I should search for stories about the Twin Empresses in the Arknights story database.
Action: answer story questions using vector db
Action Input: Twin Empresses
Observation: In Twin Empresses, the story begins with Exusiai and Mostima being reunited, but their reunion is short-lived as Mostima quickly leaves. They are then faced with a new crisis as they are surrounded by thugs in the slums. However, they manage to escape and meet up with Texas, who receives a call from Emperor. Emperor instructs them to attend a Sauin Carnival Gala and promises to make an appearance as a rap god. They agree to attend the party and prepare to hit the road.

In the Hill of Sten region, a concert is held and the older nobles praise the performance while the young nobles feel sad about lost love. They discuss the celebrations of Emperor Hou and Cui Lintmeme. Viviana and Krala have a dialogue where Cora hints that her father, who is the emperor, has died. A

{'input': 'find all stories about Twin Empresses?',
 'chat_history': '',
 'output': 'There are several stories about the Twin Empresses in the Arknights story.'}

In [5]:
agent_chain.run("write a fan fiction of Nearl passionately kissing Vivana after their match")



> Entering new AgentExecutor chain...
Thought: I need to generate a fan fiction story based on the given prompt.
Action: fan_fiction
Action Input: Nearl passionately kisses Vivana after their match
Observation: The story continues with Maria Nearl, now the talk of the town after her surprising victory over the Left-hand Knight, Tytus Topola. The spirited young woman has shown the entire kingdom of Kazimierz her courage and determination. Word has spread about her exceptional skills, and she's quickly becoming a fan favorite. However, with every victory comes a new challenge.

A new competitor steps into the arena, a woman known as Vivana. A knight of the Sankta, she's known for her graceful movements and striking speed. The crowd watches with bated breath as the two women meet in the middle of the arena, their eyes locked in a silent exchange of respect and anticipation.

The match begins with a flurry of movement. Maria's attacks are powerful, but Vivana moves like water, dodging an

'Nearl passionately kisses Vivana after their match in a moment of celebration and mutual respect.'

In [3]:
#call local api for unrestricted language model
#use the first model to query and summarize the background first, 
context = prts_query.run("Viviana and Margaret Nearl")

In [6]:
#then pass it to the local model via api
res = prts_fiction.run_api(context, "\nBased on the context above, write an yuri fan fiction of Viviana and Margaret")
print(res)

 it is the Knights and the following of the legacy of their knights of their miscreation, which includes a questionable about the cornerstone. Inquiry of Marguerite's joke, and knit, they are responsible for their sir, she is using it. This includes with their kazo, Margaret's commentary, she quotes the notion that I's not like Kaze is. However, what the opposite of them. They are notoriously: this was in their legend, which they were quidian, you may be aware of the guy, unre not. She has been that it, in a minute.


In [5]:
context

"In the NL-2 story, Maria Nearl decides to compete in the Major knight competition to save her family from bankruptcy. Despite objections from her uncle, Maria begins training under Zofia and shows improvement in her swordsmanship. Zofia warns her about the challenges she will face in the Major, but Maria remains determined to protect her family's honor. Eventually, Zofia agrees to coach Maria and allows her to compete in the Major.\n\nIn the NL-4 story, Margaret Nearl and Viviana Droste engage in a conversation about their motivations and beliefs as knights. Margaret reveals her goal of reminding the knights of Kazimierz of their forgotten glory. The story also includes a tense confrontation between Margaret and her uncle, Młynar, where he urges her to quit the tournament. However, Margaret remains determined to continue competing and fighting for her family's honor.\n\nIn the NL-4 story, Margaret Nearl and Viviana Droste engage in a conversation about their motivations and beliefs as